# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

## Team Members: Zack Bilderback, Estevan Gonzales

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

### Part 1

In [2]:
# load the data and remove duplicates
data = pd.read_csv('games.csv').drop_duplicates()

# Gets index of all numeric data and make new dataframe with that data
numeric_feats = data.dtypes[data.dtypes != "object"].index
allNumeric = data[numeric_feats]

# filling NA's with the mean of the column:
allNumeric = allNumeric.fillna(allNumeric.mean())

label_name = 'average_rating'
y = allNumeric[label_name]
X = allNumeric.drop(label_name,axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

### Part 2

In [3]:
parameters = {'n_estimators':[5, 10, 25, 50, 100, 200]}
rfr = RandomForestRegressor()
clf = GridSearchCV(rfr, parameters)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 25, 50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [4]:
print "{}: {}".format("RandomForest Best parameters", clf.best_params_)
print "{}: {}".format("RandomForest Best score", clf.best_score_)

RandomForest Best parameters: {'n_estimators': 200}
RandomForest Best score: 0.892490162827


In [5]:
# In sample RMSE
Ytrain_pred = clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('RandomForest Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('RandomForest Test RMSE', np.sqrt(Ytest_mse))

RandomForest Train RMSE: 0.368868781574
RandomForest Test RMSE: 0.993109765279


### Part 3

In [6]:
parameters = {'n_estimators':[5, 10, 25, 50, 100, 200], 'learning_rate':[1.0, 0.5, 0.1, 0.01, 0.001, 0.0001]}

adr = AdaBoostRegressor()
gbr = GradientBoostingRegressor()

adr_clf = GridSearchCV(adr, parameters)
gbr_clf = GridSearchCV(gbr, parameters)

adr_clf.fit(X_train, y_train)
gbr_clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 25, 50, 100, 200], 'learning_rate': [1.0, 0.5, 0.1, 0.01, 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [7]:
print "{}: {}".format("AdaBoost Best parameters", adr_clf.best_params_)
print "{}: {}".format("AdaBoost Best score", adr_clf.best_score_)

AdaBoost Best parameters: {'n_estimators': 50, 'learning_rate': 0.1}
AdaBoost Best score: 0.862395519354


In [8]:
print "{}: {}".format("GradientBoost Best parameters", gbr_clf.best_params_)
print "{}: {}".format("GradientBoost Best score", gbr_clf.best_score_)

GradientBoost Best parameters: {'n_estimators': 200, 'learning_rate': 0.5}
GradientBoost Best score: 0.891381377369


In [9]:
# In sample RMSE
Ytrain_pred = adr_clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('AdaBoost Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = adr_clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('AdaBoost Test RMSE', np.sqrt(Ytest_mse))

AdaBoost Train RMSE: 1.12991352282
AdaBoost Test RMSE: 1.13359305794


In [10]:
# In sample RMSE
Ytrain_pred = gbr_clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('GradientBoost Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = gbr_clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('GradientBoost Test RMSE', np.sqrt(Ytest_mse))

GradientBoost Train RMSE: 0.95195870728
GradientBoost Test RMSE: 1.00651340389


### Part 4

We thought the GradientBoostingRegressor model would be the most accurate because it is known to be very powerful and almost always performs the best out of all models so this was our initial guess. If we look at the scores of each model, the GradientBoostingRegressor and RandomForest performed the best but the RandomForest model had the best test RMSE. I think it is overfitting however since there is a big difference between the in-sample RMSE and out-of-sample RMSE. With more fine tuning, the GradientBoostingRegressor would most likely perform the best.

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score,roc_curve)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)

### Part 1

In [12]:
# load the data and remove duplicates
data = pd.read_csv('spam_uci.csv')

# last column name is '57'
label_name = '57'
y = data[label_name]
X = data.drop(label_name,axis=1)

# split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)
#X_train = X_train.values
#X_test = X_test.values
#y_train = y_train.values

# reshape for gridsearch cv's expected data shape
#c = y_train.shape
#y_train = y_train.reshape(c,)
#y_test = y_test.values

### Part 2

In [13]:
parameters = {'n_estimators':[5, 10, 15, 25, 50], 'criterion':['gini', 'entropy']}
rfc = RandomForestClassifier()
rfc_clf = GridSearchCV(rfc, parameters)
rfc_clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15, 25, 50], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [14]:
print "{}: {}".format("RandomForest Best parameters", rfc_clf.best_params_)
print "{}: {}".format("RandomForest Best score", rfc_clf.best_score_)

RandomForest Best parameters: {'n_estimators': 50, 'criterion': 'gini'}
RandomForest Best score: 1.0


In [15]:
# In sample RMSE
Ytrain_pred = rfc_clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('RandomForest Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = rfc_clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('RandomForest Test RMSE', np.sqrt(Ytest_mse))

RandomForest Train RMSE: 0.0
RandomForest Test RMSE: 0.0


In [19]:
rfc = RandomForestClassifier(n_estimators = 50, criterion = 'gini')
model = rfc.fit(X_train, y_train)
Ytest_pred = model.predict(X_test)
pred_probs = model.predict_proba(X_test)

In [20]:
def RocCurve(predprobs, actual_y):
    '''predicted probabilities should be in form of an array
    
    function returns the fpr, tpr, threshold and the AUC of the ROC Curve as the score'''
    
    pos_probs = []
    for i in predprobs:
        pos_probs.append(i[1])
    fpr, tpr, threshold = roc_curve(actual_y, pos_probs)
    score = roc_auc_score(actual_y, pos_probs)
    return fpr, tpr, threshold, score

In [21]:
fpr, tpr, threshold, score = RocCurve(pred_probs, y_test)
print "{}: {}".format('AUC_score', score)
ACC = accuracy_score(y_test, Ytest_pred) 
print "{}: {}".format('accuracy score', ACC)

AUC_score: 1.0
accuracy score: 0.999341672153


### Part 3

In [30]:
adc_parameters = {'n_estimators':[2, 5, 10, 25, 50, 100, 200], 'learning_rate':[1.0, 0.5, 0.1, 0.01, 0.001, 0.0001]}
gbc_parameters = {'n_estimators':[2, 5, 10, 25, 50, 100, 200], 'learning_rate':[1.0, 0.5, 0.1, 0.01, 0.001, 0.0001], 
                  'max_depth':[1, 2, 3, 5, 7, 10]}

adc = AdaBoostClassifier()
gbc = GradientBoostingClassifier()

adc_clf = GridSearchCV(adc, adc_parameters)
gbc_clf = GridSearchCV(gbc, gbc_parameters)

adc_clf.fit(X_train, y_train)
gbc_clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [2, 5, 10, 25, 50, 100, 200], 'learning_rate': [1.0, 0.5, 0.1, 0.01, 0.001, 0.0001], 'max_depth': [1, 2, 3, 5, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [31]:
print "{}: {}".format("AdaBoost Best parameters", adc_clf.best_params_)
print "{}: {}".format("AdaBoost Best score", adc_clf.best_score_)

AdaBoost Best parameters: {'n_estimators': 2, 'learning_rate': 1.0}
AdaBoost Best score: 1.0


In [32]:
print "{}: {}".format("GradientBoost Best parameters", gbc_clf.best_params_)
print "{}: {}".format("GradientBoost Best score", gbc_clf.best_score_)

GradientBoost Best parameters: {'n_estimators': 2, 'learning_rate': 1.0, 'max_depth': 1}
GradientBoost Best score: 1.0


In [33]:
# In sample RMSE
Ytrain_pred = adc_clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('AdaBoost Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = adc_clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('AdaBoost Test RMSE', np.sqrt(Ytest_mse))

AdaBoost Train RMSE: 0.0
AdaBoost Test RMSE: 0.0362857505715


In [34]:
# In sample RMSE
Ytrain_pred = gbc_clf.predict(X_train)
Ytrain_mse = mean_squared_error(y_train, Ytrain_pred)
print "{}: {}".format('GradientBoost Train RMSE', np.sqrt(Ytrain_mse))

# Out of sample RMSE
Ytest_pred = gbc_clf.predict(X_test)
Ytest_mse = mean_squared_error(y_test, Ytest_pred)
print "{}: {}".format('GradientBoost Test RMSE', np.sqrt(Ytest_mse))

GradientBoost Train RMSE: 0.0
GradientBoost Test RMSE: 0.0362857505715


In [38]:
adc = AdaBoostClassifier(n_estimators = 2, learning_rate = 1.0)
gbc = GradientBoostingClassifier(n_estimators = 2, learning_rate = 1.0, max_depth = 1)

adc_model = adc.fit(X_train, y_train)
gbc_model = gbc.fit(X_train, y_train)

adc_Ytest_pred = adc_model.predict(X_test)
gbc_Ytest_pred = gbc_model.predict(X_test)

adc_pred_probs = adc_model.predict_proba(X_test)
gbc_pred_probs = gbc_model.predict_proba(X_test)

In [39]:
fpr, tpr, threshold, score = RocCurve(adc_pred_probs, y_test)
print "{}: {}".format('AdaBoost AUC_score', score)
ACC = accuracy_score(y_test, adc_Ytest_pred) 
print "{}: {}".format('AdaBoost accuracy score', ACC)

AdaBoost AUC_score: 0.998871331828
AdaBoost accuracy score: 0.998683344305


In [40]:
fpr, tpr, threshold, score = RocCurve(gbc_pred_probs, y_test)
print "{}: {}".format('GradientBoost AUC_score', score)
ACC = accuracy_score(y_test, gbc_Ytest_pred) 
print "{}: {}".format('GradientBoost accuracy score', ACC)

GradientBoost AUC_score: 0.998871331828
GradientBoost accuracy score: 0.998683344305


### Part 4

One advantage of Random Forests over GBDT is that the training can be highly parallelized.  Each tree (and subtree) can be trained on independent hardware. This allows you to scale it to massive data that you would have to subsample for other techniques, and you can use it for online training where additional data is constantly streaming in. One disadvantage of Random Forests compared to GBDT is that GBDT typically outperforms Random Forests at the expense of complexity. 

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [12]:
import numpy as np
import pandas as pd
titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')